In [1]:
import rospy,tf, numpy as np, transformations, json, rospkg, yaml
from os import system
from scipy.optimize import minimize
import transformations
import time

In [2]:
rospy.init_node('calibrator')

In [3]:
rospy.set_param('/optitrack/objects', ['/robot/calibrator'])

In [4]:
tfl = tf.TransformListener(True, rospy.Duration(2))  # tf will have 2 seconds of cache

In [5]:
rospack = rospkg.RosPack()

In [6]:
world_frame_robot = rospy.get_param('/optitrack/world_frame') # The world frame of the robot
world_frame_opt = 'optitrack_frame' # The world frame of optitrack
calib_frame_robot = 'left_gripper' # The frame used for calibration in the robot's frame
calib_frame_opt = '/robot/calibrator' # The frame used for calibration in the optitrack's frame

In [8]:
print "Now run the publisher: roslaunch optitrack_publisher optitrack_publisher.launch ip:=<IP> world:={}".format(world_frame_robot)

Now run the publisher: roslaunch optitrack_publisher optitrack_publisher.launch ip:=<IP> world:=base


Also check that the ROS master and the local machine are up-to-date regarding time, to synchronize your local machine's time execute:
```
ntpdate -q ntp.ubuntu.com
```

## Definition of functions
Nothing to do here, just executing...
### Record calibration points

In [9]:
def record_calibration_points(continuous = True, duration=60, min_dist=0.01, max_dur=0.05):
    mat_robot = [] # Matrix of all calibration points of calib_frame_robot in frame world_frame_robot (position only)
    mat_opt = [] # Matrix of all calibration points of calib_frame_opt in frame world_frame_opt (position only)
    max_dur = rospy.Duration(max_dur) # seconds
    duration = rospy.Duration(duration)
    
    start = rospy.Time.now()
    last_point = None
    entry = ""
    while continuous and rospy.Time.now()<start+duration or not continuous and entry=="":   
        ref_time = tfl.getLatestCommonTime(world_frame_robot, calib_frame_opt)
        now = rospy.Time.now()
        
        if ref_time > now - max_dur:
            try:
                pose_rg_robot = tfl.lookupTransform(world_frame_robot, calib_frame_robot, rospy.Time(0))
            except Exception, e:
                print "Robot <-> Optitrack transformation not available at the last known common time:", e.message
            else:
                if last_point is None or transformations.distance(pose_rg_robot, last_point)>min_dist:
                    try:
                        pose_rg_opt = tfl.lookupTransform(world_frame_opt, calib_frame_opt, rospy.Time(0))
                    except:
                        print "Optitrack Marker not visible at the last know common time"
                    else:
                        mat_robot.append(np.array(pose_rg_robot))
                        mat_opt.append(np.array(pose_rg_opt))
                        last_point = pose_rg_robot
                        system('beep')
        else:
            print "TFs are", (now - ref_time).to_sec(), "sec late"
        
        if continuous:
            rospy.sleep(0.25)
        else:
            entry = raw_input("Press enter to record a new point or q-enter to quit ({} points)".format(len(mat_robot)))
    return mat_opt, mat_robot

### Formating functions

In [10]:
def extract_transforms(flat_transforms):
    # a transform is 3 pos and 4 rot
    nb_transform = len(flat_transforms) / 7
    list_transforms = []
    for i in range(nb_transform):
        pose = []
        # extract the pose
        pose.append(flat_transforms[i * 7:i * 7 + 3])
        pose.append(flat_transforms[i * 7 + 3:i * 7 + 7])
        # append it to the list of transforms
        list_transforms.append(pose)
    return list_transforms
        
def result_to_calibration_matrix(result):
    calibration_matrix = transformations.inverse_transform(result)
    return [map(float, calibration_matrix[0]), map(float, calibration_matrix[1].tolist())]

### Cost function for the optimization

In [11]:
def evaluate_calibration(calibrations, coords_robot, coords_opt):
    def quaternion_cost(norm_coeff):
        C = 0
        for transform in list_calibr:
            # norm of a quaternion is always 1
            C += norm_coeff * abs(np.linalg.norm(transform[1]) - 1)
        return C
        
    def distance_cost(pose1, pose2, rot_coeff=2):
        pos_cost = 0
        # calculate position ditance
        pos_cost = np.linalg.norm(np.array(pose1[0]) - np.array(pose2[0]))
        # distance between two quaternions
        rot_cost = 1 - np.inner(pose1[1], pose2[1])**2
        return pos_cost + rot_coeff * rot_cost

    # first extract the transformations
    list_calibr = extract_transforms(calibrations)
    # set the base transform
    A = list_calibr[0]
    B = list_calibr[1]
    # loop trough all the transforms
    cost = quaternion_cost(1)
    nb_points = len(coords_robot)
    for i in range(nb_points):
        robot = coords_robot[i]
        opt = coords_opt[i]
        product = transformations.multiply_transform(robot, B)
        product = transformations.multiply_transform(A, product)
        product[1] /= np.linalg.norm(product[1])
        cost += distance_cost(opt, product)
    return cost

In [12]:
bounds = []
pos_bounds = [-4, 4]
rot_bounds = [-1, 1]
for i in range(2):
    for j in range(3):
        bounds.append(pos_bounds)
    for j in range(4):
        bounds.append(rot_bounds)

### Functions for quality checking

In [13]:
def calculate_position_error(A, B, coords_robot, coords_opt):
    norm = 0.
    # precision error
    for i in range(len(coords_robot)):
        robot = coords_robot[i]
        opt = coords_opt[i]
        product = tranformations.multiply_tranform(robot, B)
        product = tranformations.multiply_tranform(A, product)
        norm += np.linalg.norm(opt[0], product[0])
    return norm

## Acquisition of points for calibration by moving the arm...

In [14]:
# Record during 60 sec... set continuous=False for an interactive mode
mat_opt, mat_robot = record_calibration_points(continuous=True)

In [15]:
print len(mat_opt), "points recorded"

113 points recorded


## Optimization of the calibration matrix

In [16]:
initial_guess = [0,0,0,0,0,0,1]*2

In [17]:
t0 = time.time()
# Be patient, this cell can be long to execute...
result = minimize(evaluate_calibration, initial_guess, args=(mat_robot, mat_opt, ),
                  method='L-BFGS-B', bounds=bounds)
print time.time()-t0, "seconds of optimization"

114.883852005 seconds of optimization


In [18]:
print result

      fun: 0.78325835213130235
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -3.65115049e-02,  -1.09724119e-02,  -5.53316393e-02,
         4.56721438e-02,   4.03179046e-02,   6.06682127e-01,
         7.07527792e-01,  -7.22845894e-02,  -7.42909401e-02,
        -6.78013201e-05,   4.88582286e-02,   6.12686546e-01,
        -2.32589770e-01,   3.92384925e-01])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 4845
      nit: 193
   status: 0
  success: True
        x: array([  1.98103156e-01,   8.22789480e-01,   9.29351936e-01,
         4.70738708e-04,  -7.01526857e-04,  -7.16430454e-01,
         6.97658005e-01,  -3.73378973e-03,   9.00575242e-04,
         1.25077429e-02,  -5.94155277e-01,   5.95435465e-01,
        -2.59463560e-01,   4.74462622e-01])


In [19]:
result_list = extract_transforms(result.x)

## Dumping A i.e. the calibration matrix (the transformation between the optitrack frame and the robot base frame)

In [20]:
calibration_matrix_a = result_to_calibration_matrix(result_list[0])

In [21]:
rospy.set_param("/optitrack/calibration_matrix", calibration_matrix_a)

In [22]:
with open(rospack.get_path("optitrack_publisher")+"/config/calibration_matrix.yaml", 'w') as f:
    yaml.dump(calibration_matrix_a, f)

## Dumping B (the transformation between the optitrack marker and the end effector frame)

In [23]:
calibration_matrix_b = result_to_calibration_matrix(result_list[1])

In [24]:
with open(rospack.get_path("optitrack_publisher")+"/config/calibration_matrix_b.yaml", 'w') as f:
    yaml.dump(calibration_matrix_b, f)

## Testing calibration quality

In [ ]:
mat_opt_check, mat_robot_check = record_calibration_points(False)

Press enter to record a new point or q-enter to quit (1 points)


In [28]:
print len(mat_opt), "points recorded"

1 points recorded


In [29]:
calculate_cost(result_a, result_b, mat_robot_check, mat_opt_check)

NameError: name 'calculate_cost' is not defined

In [30]:
avg_error = calculate_position_error(result_a, result_b, mat_robot_check, mat_opt_check)/len(mat_opt_check)

NameError: name 'result_a' is not defined

In [ ]:
avg_error